In [ ]:
import gzip
import json
import os

from elasticsearch import Elasticsearch, helpers


In [ ]:
es = Elasticsearch()
scan_query = {'query': {'match_all': {}}}
export_path = 'data'
bulk_size = 50000

os.makedirs(export_path, exist_ok=True)

In [ ]:
log_indices = es.indices.get_alias(index='logana_log').keys()

In [ ]:
def write_bulk_file(log_index, count, bulk_data): 
    bulk_path = f'{export_path}/{log_index}_{count}.ndjson.gz'
    print(f'writing {bulk_path}')
    with gzip.open(bulk_path, mode='wt', encoding='utf-8') as f:
        for data in bulk_data:
            f.write(json.dumps(data, ensure_ascii=False))
            f.write('\n')

for log_index in log_indices:
    print(f'index: {log_index}')
    docs = helpers.scan(es,query=scan_query, index=log_index)
    bulk_data = []
    count = 1
    for doc in docs:
        index = doc['_index']
        doc_id = doc['_id']
        source = doc['_source']
        bulk_data.append({'index':{'_index':index, '_id':doc_id}})
        bulk_data.append(source)
        if len(bulk_data) >= bulk_size * 2:
            write_bulk_file(log_index, count, bulk_data)
            count += 1
            bulk_data = []

    if len(bulk_data) >0:
        write_bulk_file(log_index, count, bulk_data)
    